In [1]:
import re
import time
import json
import csv
import unicodedata
import openai
import glob
import os

import numpy as np
import pandas as pd

from tqdm import tqdm
from pprint import pprint
from file_save import *

In [162]:
def is_string_in(text, phrase):
    return f' {text.lower()} ' in f' {phrase.lower()} '

def lowercase_first_letter(input_string):
    if len(input_string) > 0:
        return input_string[0].lower() + input_string[1:]
    return input_string  # Return input as is for an empty string


## Lower But consider all capitalise

In [23]:
def compute_accuracy(tuples):
    """Measure the model's accuracy."""
    n_correct = int(0)
    n_correct_substr = int(0)
    n_tuples = len(tuples)
    for t in tuples:
        #gpt_choices = t["bloom_choices"]
        gpt_choices = t["gpt3.5-turbo_choices"]
        for item in range(len(gpt_choices)):
            gpt_choices[item] = gpt_choices[item].strip()
            
        obj_labels = t["obj_label"]

        # An answer is correct if it is one of candidate answers
        n_correct += int(any([c in obj_labels for c in gpt_choices]))

        # An answer is correct if any of the candidate answers is a substr of the answer
        count = 0
        
        for c in gpt_choices:
            for o in obj_labels:
                if not o.isupper():
                    o = o.lower()
                    if o in c.lower():
                        count = 1
                        break
                else:
                    if o in c:
                        count = 1
                        break
            if count == 1:
                break
        
        n_correct_substr += count         
        #n_correct_substr += int(any([o.lower() in c.lower() for o in obj_labels for c in gpt_choices]))
        #n_correct_substr += int(any([is_string_in(o.lower(), c.lower().replace(".", " ").replace(",", " ")) for o in obj_labels for c in gpt_choices]))
    return {
        "Total number of tuples": n_tuples,
        "# of correct answers (Exact match)": n_correct,
        "% of correct answers (Exact match)": round(100 * n_correct / n_tuples, 1),
        "# of correct answers (Overlap)": n_correct_substr,
        "% of correct answers (Overlap)": round(100 * n_correct_substr / n_tuples, 1),
    }


## OCCO

In [55]:
def compute_accuracy(tuples):
    """Measure the GPT3.5 model's accuracy."""
    n_correct = int(0)
    n_correct_substr = int(0)
    n_tuples = len(tuples)
    for t in tuples:
        gpt_choices = t["gpt4_choices"]
        for item in range(len(gpt_choices)):
            gpt_choices[item] = gpt_choices[item].strip()
            
        obj_labels = t["obj_label"]

        # An answer is correct if it is one of candidate answers
        n_correct += int(any([c in obj_labels for c in gpt_choices]))

        # An answer is correct if any of the candidate answers is a substr of the answer
        count = 0
        
        for c in gpt_choices:
            for o in obj_labels:
                if o.lower() in c.lower() or c.lower() in o.lower():
                    count = 1
                    break
            if count == 1:
                break
                
        n_correct_substr += count         
        
        #n_correct_substr += int(any([is_string_in(o.lower(), c.lower().replace(".", " ").replace(",", " ")) for o in obj_labels for c in gpt_choices]))
    return {
        "Total number of tuples": n_tuples,
        "# of correct answers (Exact match)": n_correct,
        "% of correct answers (Exact match)": round(100 * n_correct / n_tuples, 1),
        "# of correct answers (Overlap)": n_correct_substr,
        "% of correct answers (Overlap)": round(100 * n_correct_substr / n_tuples, 1),
    }


## only lower

In [ ]:
def compute_accuracy(tuples):
    """Measure the GPT3.5 model's accuracy."""
    n_correct = int(0)
    n_correct_substr = int(0)
    n_tuples = len(tuples)
    for t in tuples:
        gpt_choices = t["gpt4_choices"]
        for item in range(len(gpt_choices)):
            gpt_choices[item] = gpt_choices[item].strip()
            
        obj_labels = t["obj_label"]

        # An answer is correct if it is one of candidate answers
        n_correct += int(any([c in obj_labels for c in gpt_choices]))

        # An answer is correct if any of the candidate answers is a substr of the answer
        
        n_correct_substr += int(any([o.lower() in c.lower() for o in obj_labels for c in gpt_choices]))
        #n_correct_substr += int(any([is_string_in(o.lower(), c.lower().replace(".", " ").replace(",", " ")) for o in obj_labels for c in gpt_choices]))
    return {
        "Total number of tuples": n_tuples,
        "# of correct answers (Exact match)": n_correct,
        "% of correct answers (Exact match)": round(100 * n_correct / n_tuples, 1),
        "# of correct answers (Overlap)": n_correct_substr,
        "% of correct answers (Overlap)": round(100 * n_correct_substr / n_tuples, 1),
    }


In [9]:
def compute_accuracy(tuples):
    """Measure the GPT3.5 model's accuracy."""
    n_correct = int(0)
    n_correct_substr = int(0)
    n_tuples = len(tuples)
    for t in tuples:
        gpt_choices = t["gpt3.5-turbo_choices"]
        for item in range(len(gpt_choices)):
            gpt_choices[item] = gpt_choices[item].strip()
            
        obj_labels = t["obj_label"]

        # An answer is correct if it is one of candidate answers
        n_correct += int(any([c in obj_labels for c in gpt_choices]))

        # An answer is correct if any of the candidate answers is a substr of the answer
        
        n_correct_substr += int(any([o in c for o in obj_labels for c in gpt_choices]))
        #n_correct_substr += int(any([is_string_in(o.lower(), c.lower().replace(".", " ").replace(",", " ")) for o in obj_labels for c in gpt_choices]))
    return {
        "Total number of tuples": n_tuples,
        "# of correct answers (Exact match)": n_correct,
        "% of correct answers (Exact match)": round(100 * n_correct / n_tuples, 1),
        "# of correct answers (Overlap)": n_correct_substr,
        "% of correct answers (Overlap)": round(100 * n_correct_substr / n_tuples, 1),
    }


In [34]:
#folder_path = 'C:/Users/User1/Desktop/master/dissertation/所有结果-测试用/all_result/llama2-7b-chat/result-Llama2-long/'
folder_path = 'C:/Users/User1/Desktop/master/dissertation/所有结果-测试用/all_result/gpt3.5-result/result/'
#folder_path = 'C:/Users/User1/Desktop/master/dissertation/所有结果-测试用/all_result/bloom/bloom-7b1/'

file_paths = glob.glob(folder_path + '*')
file_paths

['C:/Users/User1/Desktop/master/dissertation/所有结果-测试用/all_result/gpt3.5-result/result\\ar',
 'C:/Users/User1/Desktop/master/dissertation/所有结果-测试用/all_result/gpt3.5-result/result\\en',
 'C:/Users/User1/Desktop/master/dissertation/所有结果-测试用/all_result/gpt3.5-result/result\\es',
 'C:/Users/User1/Desktop/master/dissertation/所有结果-测试用/all_result/gpt3.5-result/result\\zh']

In [35]:
PRETICATES = ["P1412", "P1376", "P1303", "P530", "P495", "P449", "P364", "P264", "P190", "P136", "P106", "P103", "P47", "P37", "P36", "P30", "P27", "P20", "P19", "P17"]

# Get a list of all file paths in the folder
file_paths = glob.glob(folder_path + '*')
df_info = {}

#lang = ["en-ar", "en-es", "en-extend", "zh-cn-trans-extend"]
#lang = ["en-ar", "en-es", "en-extend"]
lang = ["ar", "en", "es", "zh"]
#model_name_list = ["Llama-2-7B-chat"]
model_name_list = ["chatGPT"]
#model_name_list = ["BLOOM"]
file_name_list = []
total_list = []
for i in lang:
    info = {}
    info["lang_list"] = []
    info["N_c_e"] = []
    info["P_c_e"] = []
    info["N_c_o"] = []
    info["P_c_o"] = []
    info["time_cost"] = []

    df_info[i] = info
# Iterate over the file paths and read each file
count = 0
for file_path in file_paths:
    current_lang = file_path.split("\\")[-1]
    
    file_pathss = glob.glob(file_path + '/*')
    
    for input_file_path in file_pathss:
        start = time.perf_counter()
        output_triple = load_jsonl_file(input_file_path)

        current_file_name = input_file_path.split("\\")[-1]
        
        current_file_name= current_file_name.replace(".jsonl", "")
        current_result_list = list(compute_accuracy(output_triple).values())
        
        if current_file_name not in file_name_list:
            file_name_list.append(current_file_name)
        total_list.append(current_result_list[0])

        end = time.perf_counter()
        elapsed = end - start
        df_info[current_lang]["lang_list"].append(current_lang)
        df_info[current_lang]["N_c_e"].append(current_result_list[1])
        df_info[current_lang]["P_c_e"].append(current_result_list[2])
        df_info[current_lang]["N_c_o"].append(current_result_list[3])
        df_info[current_lang]["P_c_o"].append(current_result_list[4])
        df_info[current_lang]["time_cost"].append(elapsed)
        
        columns = ['model name','file name','tuples numbers']
        data = [model_name_list,file_name_list, total_list]
for ii in lang:
    columns.extend(['language','e', 'e%', 'o', 'o%', "time cost"])
    data.append(df_info[ii]["lang_list"])
    data.append(df_info[ii]["N_c_e"])
    data.append(df_info[ii]["P_c_e"])
    data.append(df_info[ii]["N_c_o"])
    data.append(df_info[ii]["P_c_o"])
    data.append(df_info[ii]["time_cost"])
data = tuple(data)

df = pd.DataFrame(data,index=columns).T



In [37]:
df.to_excel("testt.xlsx")

## check

In [38]:
file_path = "C:/Users/User1/Desktop/master/dissertation/所有结果-测试用/testt.xlsx"
PRETICATES = []


if os.path.isfile(file_path):
    data = pd.read_excel(file_path)[:40]
    
    header = data.columns.tolist()
    correct_header_name = header[-2].split(".")[0]

    language_count = int(data.shape[1]/6)

    df_info = {}

    culture1 = data["file name"][0].split("_")[-2] + " " + data["file name"][0].split("_")[-1]
    culture2 = data["file name"][1].split("_")[-2] + " " + data["file name"][1].split("_")[-1]
    
    for i in range(len(data["file name"])):
        if i % 2 == 0:
            f_name = str(data["file name"][i])
            if len(f_name.split("_")) > 2:
                p_name = f_name.split("_")[1]
                if p_name == "":
                    p_name = f_name.split("_")[1]

                p_name = p_name.replace("P", "")
                PRETICATES.append(p_name)

    for i in range(language_count):
        info = {}

        if not i == 0:
            name = data["language" + "." + str(i)][0]
        else:
            name = data["language"][0]

        if name == "zh-cn-trans-extend":
            name = "zh"

        if name == "en-extend":
            name = "en"

        info["lang_name"] = name

        info["lang_facts"] = []
        info["lang_acc1"] = []
        info["lang_acc2"] = []     
        print(name)

        info["lang_acc1_name"] = name + " " + culture1
        info["lang_acc2_name"] = name + " " + culture2

        df_info[i] = info

    for i in range(len(data["tuples numbers"])):
        for j in range(language_count):
            num = data["tuples numbers"][i]
            if i % 2 == 0:
                df_info[j]["lang_facts"].append(num)

                if not j == 0:
                    acc = data[correct_header_name+"."+str(j)][i]
                else:
                    acc = data[correct_header_name][i]
                df_info[j]["lang_acc1"].append(acc)
            else:
                df_info[j]["lang_facts"].append(num)

                if not j == 0:
                    acc = data[correct_header_name+"."+str(j)][i]
                else:
                    acc = data[correct_header_name][i]
                df_info[j]["lang_acc2"].append(acc)

    columns = ['Preticate']
    data_df = [PRETICATES[:20]]
    for i in range(language_count):
        columns.extend([df_info[i]["lang_acc1_name"], df_info[i]["lang_acc2_name"]])
        data_df.append(df_info[i]["lang_acc1"])
        data_df.append(df_info[i]["lang_acc2"])

    data_df = tuple(data_df)

    df = pd.DataFrame(data_df,index=columns).T
    save_path = file_path.replace(file_path.split("\\")[-1], "check/" + file_path.split("\\")[-1])


ar
en
es
zh


In [39]:
df['Preticate'][:20] = df['Preticate'][:20].astype(int)
df[:20].sort_values(by='Preticate', ascending=False)

,Preticate,ar ARAB REGION,ar WESTERN COUNTRIES,en ARAB REGION,en WESTERN COUNTRIES,es ARAB REGION,es WESTERN COUNTRIES,zh ARAB REGION,zh WESTERN COUNTRIES
5,1412,88.5,97.5,96.5,97.9,94.3,93.4,85.8,83.7
4,1376,100.0,100.0,92.6,100.0,100.0,100.0,81.5,92.3
2,1303,18.6,53.4,21.4,54.2,39.4,52.0,11.3,23.9
19,530,63.6,94.7,100.0,89.5,91.7,94.7,100.0,94.7
18,495,45.8,84.8,95.0,83.7,71.3,81.2,71.2,55.8
16,449,7.1,74.8,48.4,81.3,39.2,71.8,12.2,13.2
13,364,51.7,84.2,90.9,84.0,86.2,84.0,53.1,58.5
10,264,22.3,50.2,26.5,48.9,20.1,53.2,2.2,10.1
7,190,7.5,35.9,8.8,37.7,18.1,38.5,3.6,4.3
3,136,49.6,78.2,86.8,79.7,64.1,77.2,51.4,38.6


In [28]:
df['Preticate'][:20] = df['Preticate'][:20].astype(int)
df[:20].to_excel("tttt.xlsx")

PermissionError: [Errno 13] Permission denied: 'tttt.xlsx'

In [105]:
def is_string_in(text, phrase):
    return f' {text.lower()} ' in f' {phrase.lower()} '

a = "it"
b = "bbc"
c = "we have bbc."

if is_string_in(a, b):
    print(f"'{a}' is in '{b}'")
else:
    print(f"'{a}' is not in '{b}'")

if is_string_in(b, c):
    print(f"'{b}' is in '{c}'")
else:
    print(f"'{b}' is not in '{c}'")

'it' is not in 'bbc'
'bbc' is not in 'we have bbc.'


In [4]:
PRETICATES = ["P1412", "P1376", "P1303", "P530", "P495", "P449", "P364", "P264", "P190", "P136", "P106", "P103", "P47", "P37", "P36", "P30", "P27", "P20", "P19", "P17"]

# Get a list of all file paths in the folder
file_paths = glob.glob(folder_path + '*')
df_info = {}

#lang = ["en-ar", "en-es", "en-extend", "zh-cn-trans-extend"]
#lang = ["en-ar", "en-es", "en-extend"]
lang = ["ar", "en", "es", "zh"]
#model_name_list = ["Llama-2-7B-chat"]
#model_name_list = ["chatGPT"]
model_name_list = ["BLOOM"]
file_name_list = []
total_list = []
for i in lang:
    info = {}
    info["lang_list"] = []
    info["N_c_e"] = []
    info["P_c_e"] = []
    info["N_c_o"] = []
    info["P_c_o"] = []
    info["time_cost"] = []

    df_info[i] = info
# Iterate over the file paths and read each file
count = 0
for file_path in file_paths:
    current_lang = file_path.split("\\")[-1]
    
    file_pathss = glob.glob(file_path + '/*')
    print("--------------------------------------------------------------")
    for input_file_path in file_pathss:
        start = time.perf_counter()
        output_triple = load_jsonl_file(input_file_path)

        current_file_name = input_file_path.split("\\")[-1]
        
        current_file_name= current_file_name.replace(".jsonl", "")
        
        print("current file <" , current_file_name , "> has: " , len(output_triple))
        current_result_list = list(compute_accuracy(output_triple).values())
        
        if current_file_name not in file_name_list:
            file_name_list.append(current_file_name)
        total_list.append(current_result_list[0])

        end = time.perf_counter()
        elapsed = end - start
        df_info[current_lang]["lang_list"].append(current_lang)
        df_info[current_lang]["N_c_e"].append(current_result_list[1])
        df_info[current_lang]["P_c_e"].append(current_result_list[2])
        df_info[current_lang]["N_c_o"].append(current_result_list[3])
        df_info[current_lang]["P_c_o"].append(current_result_list[4])
        df_info[current_lang]["time_cost"].append(elapsed)
        
        columns = ['model name','file name','tuples numbers']
        data = [model_name_list,file_name_list, total_list]
for ii in lang:
    columns.extend(['language','e', 'e%', 'o', 'o%', "time cost"])
    data.append(df_info[ii]["lang_list"])
    data.append(df_info[ii]["N_c_e"])
    data.append(df_info[ii]["P_c_e"])
    data.append(df_info[ii]["N_c_o"])
    data.append(df_info[ii]["P_c_o"])
    data.append(df_info[ii]["time_cost"])
data = tuple(data)

df = pd.DataFrame(data,index=columns).T



--------------------------------------------------------------
current file < _P103_general_ARAB_REGION > has:  1000
current file < _P103_general_WESTERN_COUNTRIES > has:  1000
current file < _P106_general_ARAB_REGION > has:  1000
current file < _P106_general_WESTERN_COUNTRIES > has:  1000
current file < _P1303_general_ARAB_REGION_output > has:  1000
current file < _P1303_general_WESTERN_COUNTRIES_output > has:  1000
current file < _P136_general_ARAB_REGION_output > has:  452
current file < _P136_general_WESTERN_COUNTRIES_output > has:  1000
current file < _P1376_general_ARAB_REGION_output > has:  24
current file < _P1376_general_WESTERN_COUNTRIES_output > has:  26
current file < _P1412_general_ARAB_REGION_output > has:  1000
current file < _P1412_general_WESTERN_COUNTRIES_output > has:  1000
current file < _P17_general_ARAB_REGION > has:  1000
current file < _P17_general_WESTERN_COUNTRIES > has:  1000
current file < _P190_general_ARAB_REGION_output > has:  67
current file < _P190_gene

current file < _P530_general_ASIA_output > has:  13
current file < _P530_general_WESTERN_COUNTRIES_output > has:  19


## Find the same obj

In [8]:
path1 = "C:/Users/User1/Desktop/master/dissertation/所有结果-测试用/dataset/dlama-v1/test/en-extend/"
path2 = "C:/Users/User1/Desktop/master/dissertation/所有结果-测试用/dataset/dlama-v1/test/en-es/"
path3 = "C:/Users/User1/Desktop/master/dissertation/所有结果-测试用/dataset/dlama-v1/test/en-ar/"

file_paths1 = glob.glob(path1 + '*')
file_paths2 = glob.glob(path2 + '*')
file_paths3 = glob.glob(path3 + '*')

In [21]:
subs1_all = []
subs2_all = []
subs3_all = []
for i in range(len(file_paths1)):
    predicate = file_paths1[i].split("\\")[-1]
    subs1 = [a["sub_label"] for a in load_jsonl_file(file_paths1[i])]
    subs1_all.extend(subs1)
    subs2 = [a["sub_label"] for a in load_jsonl_file(file_paths2[i])]
    subs2_all.extend(subs2)
    subs3 = [a["sub_label"] for a in load_jsonl_file(file_paths3[i])]
    subs3_all.extend(subs3)
    
    common_elements = set(subs1) & set(subs2) 
    num_common_elements = len(common_elements)
    print(len(subs1))
    print(len(subs2))
    print(len(subs3))
    print("for Predicate: "+predicate+ "， it has " + str(num_common_elements) + " same subjects")
    
print(len(subs1_all))
print(len(subs2_all))
print(len(subs3_all))
common_elements = set(subs1_all) & set(subs2_all) 
num_common_elements = len(common_elements)
print("for Predicate: "+predicate+ "， it has " + str(num_common_elements) + " same subjects")
    

1000
1000
1000
for Predicate: P103_general_WESTERN_COUNTRIES.jsonl， it has 840 same subjects
1000
1000
1000
for Predicate: P106_general_WESTERN_COUNTRIES.jsonl， it has 555 same subjects
1000
1000
1000
for Predicate: P1303_general_WESTERN_COUNTRIES.jsonl， it has 749 same subjects
1000
1000
1000
for Predicate: P136_general_WESTERN_COUNTRIES.jsonl， it has 783 same subjects
26
26
26
for Predicate: P1376_general_WESTERN_COUNTRIES.jsonl， it has 26 same subjects
1000
1000
1000
for Predicate: P1412_general_WESTERN_COUNTRIES.jsonl， it has 512 same subjects
1000
1000
1000
for Predicate: P17_general_WESTERN_COUNTRIES.jsonl， it has 605 same subjects
467
465
468
for Predicate: P190_general_WESTERN_COUNTRIES.jsonl， it has 454 same subjects
1000
1000
1000
for Predicate: P19_general_WESTERN_COUNTRIES.jsonl， it has 304 same subjects
1000
1000
1000
for Predicate: P20_general_WESTERN_COUNTRIES.jsonl， it has 258 same subjects
1000
1000
1000
for Predicate: P264_general_WESTERN_COUNTRIES.jsonl， it has 673 s

# find the wrong obj

In [35]:
path = "C:/Users/User1/Desktop/master/dissertation/所有结果-测试用/gpt/gpt-4/en-es/_P27_general_WESTERN_COUNTRIES_output.jsonl"
tuples = load_jsonl_file(path)
false_tuples = []

n_correct = 0
n_correct_substr = 0
n_tuples = len(tuples)
for t in tuples:
    gpt_choices = t["gpt4_choices"]
    for item in range(len(gpt_choices)):
        gpt_choices[item] = gpt_choices[item].strip()

    obj_labels = t["obj_label"]

    # An answer is correct if it is one of candidate answers
    is_correct = False
    for c in gpt_choices:
        for o in obj_labels:
            if o.lower() in c.lower():
                is_correct = True
    if is_correct == False:
        false_tuples.append(t)
        
    n_correct += int(any([c in obj_labels for c in gpt_choices]))

    # An answer is correct if any of the candidate answers is a substr of the answer
    n_correct_substr += int(any([o in c for o in obj_labels for c in gpt_choices]))
    

print(len(false_tuples))
    
print(false_tuples[3])

231
{'sub_label': 'Daniel H. Overmyer', 'obj_label': ['the U.S.', 'the U.S.A.', 'United States of America', 'Merica', 'US of A', 'the US', 'US', 'the US of A', 'United States', 'USA', 'the U.S. of America', 'the United States', 'U. S. A.', 'U.S.A.', 'U.S.', 'U. S.', 'America', 'the US of America', 'the USA', 'the States', 'the U.S. of A', 'US of America', 'U.S. of America', 'the United States of America'], 'gpt4_choices': ['canada']}


In [15]:
print(false_tuples[4])

{'sub_label': 'Maurice Duplessis', 'obj_label': ['Schefferville', 'Quebec, Canada', 'PQ', 'Quebec', 'P.Q.', 'Canada', 'CA', 'Dominion of Canada', 'CA-QC', 'CAN', 'QC', 'Québec', 'Quebec Province', 'British North America', 'Province of Quebec'], 'gpt4_choices': ['montreal']}


In [32]:
false_tuples3 = false_tuples #ar

In [34]:
false_tuples2 = false_tuples # en

In [36]:
false_tuples1 = false_tuples #es

In [42]:
same_tuples = []
count = 0
for t in false_tuples2:
    for t2 in false_tuples1:
        
        
        if t['sub_label'] == t2['sub_label']:
            print(t)
            print(t2)
            print("\n")
            same_tuples.append(t)
            count += 1

print(count)

            

{'sub_label': 'Charles III', 'obj_label': ['United Kingdom of Great Britain and Northern Ireland', 'GB', 'The United Kingdom of Great Britain and Northern Ireland', 'G. B.', 'U.K.', 'UK', 'United Kingdom', 'G B', 'Great Britain', 'GBR', 'Great Britain and Northern Ireland', '🇬🇧', 'Britain', 'G B R', 'The UK', 'U. K.', 'U K', 'G.B.', 'G.B.R.'], 'gpt4_choices': ['monaco']}
{'sub_label': 'Charles III', 'obj_label': ['G B R', 'U. K.', 'GBR', 'United Kingdom', 'Britain', 'G B', 'United Kingdom of Great Britain and Northern Ireland', 'UK', 'Great Britain and Northern Ireland', 'G.B.R.', 'U K', 'G.B.', 'The UK', '🇬🇧', 'The United Kingdom of Great Britain and Northern Ireland', 'U.K.', 'G. B.', 'GB', 'Great Britain'], 'gpt4_choices': ['monaco']}


{'sub_label': 'Frédéric Chopin', 'obj_label': ['Tsarist Russia', 'Moscovy', 'Moscovy or Russia', 'Empire of Russia', 'Congress Poland', 'Kingdom of Poland', 'Second Republic of France', 'Duchy of Warsaw', 'Rusia', 'Russian Poland', 'Tsardom of Poland

In [40]:
print(len(false_tuples1)) #
print(len(false_tuples2)) # 1-2: 9
print(len(false_tuples3)) #1-3: 34  2-3: 10

231
15
56


## Update obj_label

In [3]:
need_change_folder_path = 'C:/Users/User1/Desktop/master/dissertation/所有结果-测试用/BLOOM/result-bloom/zh-cn-trans-extend/'

original_folder_path = "C:/Users/User1/Desktop/master/dissertation/所有结果-测试用/dataset/dlama-v1/asia-west/dlama/zh-cn-trans/"

PRETICATES = ["P1412", "P1376", "P1303", "P530", "P495", "P449", "P364", "P264", "P190", "P136", "P106", "P103", "P47", "P37", "P36", "P30", "P27", "P20", "P19", "P17"]

save_folder_path = "C:/Users/User1/Desktop/master/dissertation/所有结果-测试用/BLOOM/result-bloom/修复的result/"

need_change_file_paths = glob.glob(need_change_folder_path + '*')
need_change_file_paths[0].split("\\")[-1]

'bloom-1b1_P103_general_ASIA_output.jsonl'

In [11]:
for file in need_change_file_paths:
    #for bloom
    model_name = file.split("\\")[-1].split("_")[0]
    #######
    need_change_triples = load_jsonl_file(file)
    
    current_file_name = file.split("\\")[-1]
   
    output_path = save_folder_path + "zh-cn-trans-extend/" + current_file_name
    only_change_output_path = save_folder_path + "change/" + current_file_name
    
    current_file_name = current_file_name.replace(model_name+"_", "").replace("_output", "")
    
    original_triples = load_jsonl_file(original_folder_path + current_file_name)
    
    output_triples = []
    only_changed_triples = []
    
    for triple in need_change_triples:
        if len(triple["obj_label"]) == 0:
            for ori_triple in original_triples:
                if ori_triple["sub_label"] == triple["sub_label"]:
                    triple["obj_label"] = ori_triple["obj_label"]
                    triple["Modify_obj?"] = "Yes"
                    only_changed_triples.append(triple)
                    
        output_triples.append(triple)
    
    save_jsonl_file_no_EN(output_path, output_triples)    
    save_jsonl_file_no_EN(only_change_output_path, only_changed_triples)    
            
            
        
    
        

## Calculate Accuracy

In [7]:
save_folder_path = "C:/Users/User1/Desktop/master/dissertation/所有结果-测试用/BLOOM/result-bloom/修复的result/zh-cn-trans-extend/"

file_paths = glob.glob(save_folder_path + '*')


df_info = {}

lang = ["zh-cn-trans-extend"]
model_name_list = []
file_name_list = []
total_list = []
for i in lang:
    info = {}
    info["lang_list"] = []
    info["N_c_e"] = []
    info["P_c_e"] = []
    info["N_c_o"] = []
    info["P_c_o"] = []
    info["time_cost"] = []

    df_info[i] = info
# Iterate over the file paths and read each file
count = 0
columns = ['model name','file name','tuples numbers']
data = []
for file_path in file_paths:
    current_lang = "zh-cn-trans-extend"
    
    #for bloom
    model_name = file_path.split("\\")[-1].split("_")[0]
    model_name_list.append(model_name)
    #######
    
    start = time.perf_counter()
    output_triple = load_jsonl_file(file_path)
    current_file_name = file_path.split("\\")[-1]

    current_file_name= current_file_name.replace(".jsonl", "")
    current_result_list = list(compute_accuracy(output_triple).values())

    file_name_list.append(current_file_name)
    total_list.append(current_result_list[0])

    end = time.perf_counter()
    elapsed = end - start
    df_info[current_lang]["lang_list"].append(current_lang)
    df_info[current_lang]["N_c_e"].append(current_result_list[1])
    df_info[current_lang]["P_c_e"].append(current_result_list[2])
    df_info[current_lang]["N_c_o"].append(current_result_list[3])
    df_info[current_lang]["P_c_o"].append(current_result_list[4])
    df_info[current_lang]["time_cost"].append(elapsed)

    columns = ['model name','file name','tuples numbers']
    data = [model_name_list,file_name_list, total_list]
        
for ii in lang:
    columns.extend(['language','e', 'e%', 'o', 'o%', "time cost"])
    data.append(df_info[ii]["lang_list"])
    data.append(df_info[ii]["N_c_e"])
    data.append(df_info[ii]["P_c_e"])
    data.append(df_info[ii]["N_c_o"])
    data.append(df_info[ii]["P_c_o"])
    data.append(df_info[ii]["time_cost"])
data = tuple(data)

df = pd.DataFrame(data,index=columns).T
df.to_excel("C:/Users/User1/Desktop/master/dissertation/所有结果-测试用/BLOOM/result-bloom/修复的result/test.xlsx")
        
    
    

In [26]:
load_jsonl_file(input_file_path)

[{'sub_uri': 'Q39',
  'obj_uri': ['Q1016',
   'Q1246',
   'Q142',
   'Q145',
   'Q148',
   'Q159',
   'Q159583',
   'Q16',
   'Q183',
   'Q184',
   'Q217',
   'Q218',
   'Q227',
   'Q252',
   'Q258',
   'Q28',
   'Q30',
   'Q347',
   'Q35',
   'Q38',
   'Q399',
   'Q40',
   'Q403',
   'Q408',
   'Q41',
   'Q414',
   'Q423',
   'Q43',
   'Q458',
   'Q77',
   'Q79',
   'Q794',
   'Q801',
   'Q833',
   'Q843',
   'Q854',
   'Q865',
   'Q884',
   'Q928',
   'Q96'],
  'sub_label': 'Switzerland',
  'obj_label': ['ID',
   'Taiwan',
   'Chunghwa Minkwo',
   'State of Libya',
   'el',
   'Egypt',
   'United States of America',
   'France',
   'European Union',
   'Bielaruś',
   'md',
   'Republic of China (Taiwan)',
   'Federation of Malaysia',
   '🇱🇾',
   'az',
   'Hellas',
   'Belarus',
   'Chair of Peter',
   'the US of America',
   'Taprobane',
   'Europe',
   'Korea (South)',
   'N. Korea',
   'eu',
   'Azərbaycan Respublikası',
   'Rep. Egypt',
   'id',
   '🇱🇮',
   'U. S. A.',
   'US of M

In [ ]:
input_file_path = "dlama-v1\en-es\P449_general_WESTERN_COUNTRIES_output.jsonl"
tuples = load_jsonl_file(input_file_path)
print(compute_accuracy(tuples))